In [15]:
# Importing librairies
import sqlite3
import pandas as pd
from datetime import datetime
from sqlalchemy import create_engine
from sqlalchemy.types import FLOAT, TEXT, INTEGER, DATE

In [6]:
# Dataset Import
df = pd.read_csv("../Transform/weather_data_transformed.csv")
df

,latitude,longitude,date,time_of_day,temperature_2m,relative_humidity_2m,precipitation,wind_speed_10m,weather_code,location,heat_index,is_raining
0,43.2965,5.3698,2025-02-25,Night,-1.339673,0.018646,0.085135,0.542909,0.0,Marseille 02,-0.951896,1
1,43.2965,5.3698,2025-02-25,Morning,-1.500753,0.101450,0.085135,0.547789,0.0,Marseille 02,-1.042330,1
2,43.2965,5.3698,2025-02-25,Afternoon,-1.346172,0.123200,0.085135,0.542316,0.0,Marseille 02,-0.955544,1
3,43.2965,5.3698,2025-02-25,Evening,-1.164595,0.032979,0.085135,0.536627,0.0,Marseille 02,-0.853602,1
4,43.2965,5.3698,2025-02-26,Night,-1.328588,0.014805,0.085135,0.542530,0.0,Marseille 02,-0.945672,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1483,48.8566,2.3522,2025-05-29,Evening,1.976579,-0.926833,-0.239521,-0.347950,0.0,Paris,2.734410,0
1484,48.8566,2.3522,2025-05-30,Night,0.847264,0.952639,-0.239521,-1.506526,0.0,Paris,0.275907,0
1485,48.8566,2.3522,2025-05-30,Morning,0.888834,0.802281,-0.239521,-1.180385,0.0,Paris,0.299245,0
1486,48.8566,2.3522,2025-05-30,Afternoon,2.828761,-1.270507,-0.239521,-0.686515,0.0,Paris,3.007469,0


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1488 entries, 0 to 1487
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   latitude              1488 non-null   float64
 1   longitude             1488 non-null   float64
 2   date                  1488 non-null   object 
 3   time_of_day           1488 non-null   object 
 4   temperature_2m        1488 non-null   float64
 5   relative_humidity_2m  1488 non-null   float64
 6   precipitation         1488 non-null   float64
 7   wind_speed_10m        1488 non-null   float64
 8   weather_code          1488 non-null   float64
 9   location              1488 non-null   object 
 10  heat_index            1488 non-null   float64
 11  is_raining            1488 non-null   int64  
dtypes: float64(8), int64(1), object(3)
memory usage: 139.6+ KB


In [21]:
# Convertir les colonnes catégoriques
df['time_of_day'] = df['time_of_day'].astype('category')
df['location'] = df['location'].astype('category')
df['weather_code'] = df['weather_code'].astype(str)  # Convertir float64 en chaîne pour codes météo
df['date'] = pd.to_datetime(df['date']).dt.date  # Convertir en datetime.date
df['is_raining'] = df['is_raining'].astype('int32')  # Optimiser en int32

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1488 entries, 0 to 1487
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   latitude              1488 non-null   float64 
 1   longitude             1488 non-null   float64 
 2   date                  1488 non-null   object  
 3   time_of_day           1488 non-null   category
 4   temperature_2m        1488 non-null   float64 
 5   relative_humidity_2m  1488 non-null   float64 
 6   precipitation         1488 non-null   float64 
 7   wind_speed_10m        1488 non-null   float64 
 8   weather_code          1488 non-null   object  
 9   location              1488 non-null   category
 10  heat_index            1488 non-null   float64 
 11  is_raining            1488 non-null   int32   
 12  location_id           1488 non-null   int64   
dtypes: category(2), float64(7), int32(1), int64(1), object(2)
memory usage: 125.5+ KB


In [2]:
# Creating and connecting to SQLite database
conn = sqlite3.connect('weather_database.db')
cursor = conn.cursor()

In [3]:
# Creating Locations table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Locations (
        location_id INTEGER PRIMARY KEY AUTOINCREMENT,
        latitude FLOAT NOT NULL,
        longitude FLOAT NOT NULL,
        location TEXT NOT NULL
    )
''')

In [4]:
# Creating Weather_Observations table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Weather_Observations (
        observation_id INTEGER PRIMARY KEY AUTOINCREMENT,
        location_id INTEGER NOT NULL,
        date DATE NOT NULL,
        time_of_day TEXT NOT NULL,
        temperature_2m FLOAT NOT NULL,
        relative_humidity_2m FLOAT NOT NULL,
        precipitation FLOAT NOT NULL,
        wind_speed_10m FLOAT NOT NULL,
        weather_code TEXT NOT NULL,
        heat_index FLOAT NOT NULL,
        is_raining INTEGER NOT NULL,
        FOREIGN KEY (location_id) REFERENCES Locations(location_id)
    )
''')

In [16]:
# Création du moteur SQLite
engine = create_engine('sqlite:///weather_database.db')

# Insertion des emplacements uniques dans la table locations
locations = df[['latitude', 'longitude', 'location']].drop_duplicates()
locations.to_sql('Locations', engine, if_exists='append', index=False, dtype={
    'latitude': FLOAT,
    'longitude': FLOAT,
    'location': TEXT
})

4

In [17]:
# Récupération des location_id pour mapper
cursor.execute('SELECT location_id, latitude, longitude FROM Locations')
location_map = {(row[1], row[2]): row[0] for row in cursor.fetchall()}

In [18]:
# Ajout de location_id au DataFrame
df['location_id'] = df.apply(
    lambda row: location_map.get((row['latitude'], row['longitude'])), axis=1
)

In [19]:
df

,latitude,longitude,date,time_of_day,temperature_2m,relative_humidity_2m,precipitation,wind_speed_10m,weather_code,location,heat_index,is_raining,location_id
0,43.2965,5.3698,2025-02-25,Night,-1.339673,0.018646,0.085135,0.542909,0.0,Marseille 02,-0.951896,1,1
1,43.2965,5.3698,2025-02-25,Morning,-1.500753,0.101450,0.085135,0.547789,0.0,Marseille 02,-1.042330,1,1
2,43.2965,5.3698,2025-02-25,Afternoon,-1.346172,0.123200,0.085135,0.542316,0.0,Marseille 02,-0.955544,1,1
3,43.2965,5.3698,2025-02-25,Evening,-1.164595,0.032979,0.085135,0.536627,0.0,Marseille 02,-0.853602,1,1
4,43.2965,5.3698,2025-02-26,Night,-1.328588,0.014805,0.085135,0.542530,0.0,Marseille 02,-0.945672,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1483,48.8566,2.3522,2025-05-29,Evening,1.976579,-0.926833,-0.239521,-0.347950,0.0,Paris,2.734410,0,4
1484,48.8566,2.3522,2025-05-30,Night,0.847264,0.952639,-0.239521,-1.506526,0.0,Paris,0.275907,0,4
1485,48.8566,2.3522,2025-05-30,Morning,0.888834,0.802281,-0.239521,-1.180385,0.0,Paris,0.299245,0,4
1486,48.8566,2.3522,2025-05-30,Afternoon,2.828761,-1.270507,-0.239521,-0.686515,0.0,Paris,3.007469,0,4


In [23]:
# Insertion des observations dans Weather_Observations
weather_cols = ['location_id', 'date', 'time_of_day', 'temperature_2m', 'relative_humidity_2m',
                'precipitation', 'wind_speed_10m', 'weather_code', 'heat_index', 'is_raining']
df[weather_cols].to_sql('Weather_Observations', engine, if_exists='append', index=False, dtype={
    'location_id': INTEGER,
    'date': DATE,
    'time_of_day': TEXT,
    'temperature_2m': FLOAT,
    'relative_humidity_2m': FLOAT,
    'precipitation': FLOAT,
    'wind_speed_10m': FLOAT,
    'weather_code': TEXT,
    'heat_index': FLOAT,
    'is_raining': INTEGER
})

1488

In [24]:
print("Locations:")
print(pd.read_sql_query("SELECT * FROM Locations LIMIT 5", conn))
print("\nWeather Observations:")
print(pd.read_sql_query("SELECT * FROM Weather_Observations LIMIT 5", conn))

Locations:
   location_id  latitude  longitude      location
0            1   43.2965     5.3698  Marseille 02
1            2   44.8378    -0.5792      Bordeaux
2            3   45.7640     4.8357          Lyon
3            4   48.8566     2.3522         Paris

Weather Observations:
   observation_id  location_id        date time_of_day  temperature_2m  \
0               1            1  2025-02-25       Night       -1.339673   
1               2            1  2025-02-25     Morning       -1.500753   
2               3            1  2025-02-25   Afternoon       -1.346172   
3               4            1  2025-02-25     Evening       -1.164595   
4               5            1  2025-02-26       Night       -1.328588   

   relative_humidity_2m  precipitation  wind_speed_10m weather_code  \
0              0.018646       0.085135        0.542909          0.0   
1              0.101450       0.085135        0.547789          0.0   
2              0.123200       0.085135        0.542316    

In [25]:
conn.close()